## Importing

In [1]:
import os
import numpy as np
import cv2
import multiprocessing as mp
import time
import re
min_confidence = 0.2
size = 6000

## for renaming the file (optional)

In [ ]:
%%time
path = './images/'
i = 0
for filename in os.listdir(path):
    os.rename(os.path.join(path,filename), os.path.join(path,'image_'+str(i)+'.jpg'))
    i = i +1

## class define and model load

In [2]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel")

## function for detection

In [3]:
def prep(img,q):
    #image load
    image = cv2.imread(img)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    index = re.sub("\D", "", img)
    q.put([blob,image,index])
    return None

def detect(queue):

    arr, image, num = queue.get()
    (h, w) = image.shape[:2]
    #load image in model
    net.setInput(arr)
    detections = net.forward()

    #look for match in image
    for i in np.arange(0, detections.shape[2]):

        #extract confidence from detected object
        confidence = detections[0, 0, i, 2]

        #filter out low confidence objects
        if confidence > min_confidence:

            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
#             print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)

            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output image
    cv2.imwrite("./outpu/{}_output.jpg".format(num), image)
    return None

# Parallel

In [4]:
qw = ["./images/image_{}.jpg".format(i) for i in range(size)]
m = mp.Manager()
q = m.Queue(500)

## Parallel with 5 processors

In [5]:
def parallel_prep(qw,q,a,b):
    for i in range(a,b):
        prep(qw[i],q)

In [6]:
def parallel_detect(q,a,b):
    for i in range(a,b):
        detect(q)

In [7]:
import functools

from multiprocessing import Pool

def smap(f):
    return f()

k = size #no of images

prep1 = functools.partial(parallel_prep, qw, q, 0, k//2)
prep2 = functools.partial(parallel_prep, qw, q, k//2, k)
dete1 = functools.partial(parallel_detect, q, 0, k//3)
dete2 = functools.partial(parallel_detect, q, k//3, 2*(k//3))
dete3 = functools.partial(parallel_detect, q, 2*(k//3), k)
pool = Pool(processes=5)

In [20]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 1 processor for preprocessing and 1 processor for detection\n")
res = pool.map_async(smap, [prep1, dete1])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 1 processor for preprocessing and 1 processor for detection

CPU times: user 12.1 ms, sys: 13.6 ms, total: 25.7 ms
Wall time: 6min 4s


In [10]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 1 processor for preprocessing and 2 processors for detection\n")
res = pool.map_async(smap, [prep1, dete1, dete2])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 1 processor for preprocessing and 2 processors for detection

CPU times: user 13.4 ms, sys: 8.26 ms, total: 21.7 ms
Wall time: 3min 52s


In [8]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 2 processor for preprocessing and 2 processors for detection\n")
res = pool.map_async(smap, [prep1, prep2, dete1, dete2])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 2 processor for preprocessing and 2 processors for detection

CPU times: user 22.5 ms, sys: 4 ms, total: 26.5 ms
Wall time: 3min 29s


In [10]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 2 processor for preprocessing and 3 processors for detection\n")
res = pool.map_async(smap, [prep1, prep2, dete1, dete2, dete3])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 2 processor for preprocessing and 3 processors for detection

CPU times: user 21.9 ms, sys: 7.18 ms, total: 29.1 ms
Wall time: 3min 48s


In [8]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 3 processor for preprocessing and 2 processors for detection\n")
res = pool.map_async(smap, [prep1, prep2, prep3, dete1, dete2])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 3 processor for preprocessing and 2 processors for detection

CPU times: user 20 ms, sys: 10.2 ms, total: 30.2 ms
Wall time: 3min 30s


In [8]:
%%time
print("           Parallel Implementation of 6000 images")
print("using 2 processor for preprocessing and 1 processors for detection\n")
res = pool.map_async(smap, [prep1, prep2, dete1])
pool.close()
pool.join()

           Parallel Implementation of 6000 images
using 2 processor for preprocessing and 1 processors for detection

CPU times: user 16.7 ms, sys: 2.94 ms, total: 19.7 ms
Wall time: 4min 6s


In [66]:
while(q.empty() != True):
    q.get()

In [11]:
q.qsize()

0